# Adding labeled masks from CVAT to the original railsem images

- you need to parse the bboxes coordinates
- parse masks inside bboxes
- you need to transfer the mask using the bboxes coordinates to the original image and so on for all images

In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import shapely.geometry as geom

import json

import os
from pathlib import Path
import shutil

from pprint import pprint
from tqdm import tqdm
import locale

import shapely.geometry as geom
import numpy as np
from ultralytics.data.utils import polygon2mask


- reading json files
- extracting coordinates from rs
- coco to mask format polygon check

In [ ]:
rs_annot = 'annotation_rs/jsons/rs19_val/rs07854.json'
classnames = ['switch-left', 'switch-right', 'switch-unknow']
coco_annot = 'annotation_mine/instance-443big.json'
coco_result_annot = 'results/annot_coco_big.json'
rs_true_img_dir = 'rs_true/jpgs/rs19_val'


rs_annot_dir = 'annotation_rs/jsons/rs19_val'
rs_annot_list = os.listdir('annotation_rs/jsons/rs19_val')
rs_imgs = os.listdir(rs_true_img_dir)

In [ ]:
#create a dictionary of file names and coordinates bboxes true

bboxes_scale_dict = {}
img_names = []
for filename in rs_annot_list:
    with open(rs_annot_dir + '/' + filename) as file:
        data = json.loads(file.read())

    increase_bboxes = 2

    wh_inc = 1 + increase_bboxes / 100
    xy_inc = 1 - increase_bboxes / 100

    for idx, obj in enumerate(data['objects']):
        if obj['label'] in classnames:
            try:
                x, y, w, h = obj['boundingbox']

                if increase_bboxes != 0:
                    x, y = int(xy_inc * x), int(xy_inc * y)
                    w, h = int(wh_inc * w), int(wh_inc * h)

                    firstname = filename.split(".")[0]
                    name = f'{firstname}_{idx}'
                    img_names.append(firstname)
                    bboxes_scale_dict[name] = (x, y)
            except:
                continue


print(len(bboxes_scale_dict))

In [ ]:
bboxes_scale_dict['rs05840_10']

In [ ]:
#we get the polygons by the file name rs_true in the coco annotation
with open(coco_annot) as file:
    data = file.read()
dic = json.loads(data)


polygons = {}
for img_name in img_names:
    img_ids = []
    for img_obj in dic['images']:
        if img_name in img_obj['file_name']:
            # img_ids.append(img_obj['id'])
            for cur_annot in dic['annotations']:
                # for id in img_ids:
                if cur_annot['image_id'] == img_obj['id']:
                    polygons[img_obj['file_name'].split('.')[0]] = cur_annot['segmentation'][0]

len(polygons)

In [ ]:
#scaling polygons

result_bboxes = {}
for key, value in polygons.items():
    polygon_scaled = []
    for idx in range(len(value)):
        if idx % 2 == 0:
            try:
                polygon_scaled.append(value[idx] + bboxes_scale_dict[key][0])   #посмотреть на обучении мб придется поменять местами 1 и 0
            except:
                continue
        else:
            try:
                polygon_scaled.append(value[idx] + bboxes_scale_dict[key][1])
            except:
                continue
    result_bboxes[key] = polygon_scaled

In [ ]:
len(result_bboxes)

In [ ]:
#create new names and write all the frame polygons to one image

with open(coco_annot) as file:
    data = file.read()
dic = json.loads(data)


new_img_names = []
for item in polygons.keys():
    new_img_names.append(item.split('_')[0])

new_img_names = set(new_img_names)
len(new_img_names)

In [ ]:
WIDTH = 1920
HEIGHT = 1080

with open(coco_result_annot) as file:
    data = file.read()
dic = json.loads(data)

images = []


for idx, img_name in enumerate(new_img_names):
    images.append(
        {
            "id": idx,
            "width": WIDTH,
            "height": HEIGHT,
            "file_name": img_name + '.jpg',
            "license": 0,
            "flickr_url": "",
            "coco_url": "",
            "date_captured": 0
        },
    )
dic['images'] = images

# Adding the new bounding boxes to annotations

In [ ]:
#Adding

bboxes = {}
img_names = []
for filename in rs_annot_list:
    with open(rs_annot_dir + '/' + filename) as file:
        data = json.loads(file.read())

    increase_bboxes = 2

    wh_inc = 1 + increase_bboxes / 100
    xy_inc = 1 - increase_bboxes / 100

    for idx, obj in enumerate(data['objects']):
        if obj['label'] in classnames:
            try:
                x, y, w, h = obj['boundingbox']

                if increase_bboxes != 0:
                    x, y = int(xy_inc * x), int(xy_inc * y)
                    w, h = int(wh_inc * w), int(wh_inc * h)

                    firstname = filename.split(".")[0]
                    name = f'{firstname}_{idx}'
                    img_names.append(firstname)
                    bboxes[name] = [x, y, w, h]
            except:
                continue


pprint(bboxes)

In [ ]:
annotations = []
for idx, img_name in enumerate(polygons.keys()):
    #frame_10
    for img_object in dic['images']:
        if img_name.split('_')[0] in img_object['file_name']:
            mask = result_bboxes.get(img_name)
            bbox = bboxes.get(img_name)
            annotations.append(
                        {
                            "id": idx,
                            "image_id": img_object['id'],
                            "category_id": 1,
                            "segmentation": [
                                mask
                            ],
                            "area": 2195.0,
                            "bbox": bbox,
                            "iscrowd": 0,
                            "attributes": {
                                "occluded": False
                            }
                        },
            )

In [ ]:
len(annotations)

In [ ]:
dic['annotations'] = annotations

In [ ]:
with open(r"C:\Users\ivano\Документы\GitHub\Rail_segm\code\data_prepare\results\annot_coco_finally.json", "w", encoding="utf-8") as file:
    json.dump(dic, file)